In [ ]:
!git clone https://github.com/NetAppDocs/bluexp-automation.git


Cloning into 'bluexp-automation'...
remote: Enumerating objects: 5063, done.
remote: Counting objects: 100% (694/694), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 5063 (delta 432), reused 549 (delta 369), pack-reused 4369 (from 1)
Receiving objects: 100% (5063/5063), 2.78 MiB | 6.62 MiB/s, done.
Resolving deltas: 100% (3425/3425), done.


In [ ]:
cd bluexp-automation


/content/bluexp-automation


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.7 MB/s eta 0:00:00


In [ ]:
!find . -type f -name "*.adoc" | wc -l


158


In [ ]:
import os
print(os.getcwd())


/content/bluexp-automation


In [ ]:
!find . -name "*.adoc"

./cm/wf_common_ontap_repl_get_rel_status.adoc
./cm/wf_gcp_cloud_md_get_vpcs.adoc
./cm/wf_aws_ontap_modify_volume.adoc
./cm/wf_common_identity_get_supported_srv.adoc
./cm/wf_azure_cloud_delete_we.adoc
./cm/wf_aws_ontap_add_disks_aggr.adoc
./cm/wf_aws_cloud_md_create_account.adoc
./cm/wf_common_ontap_repl_get_rel_we_status.adoc
./cm/prepare.adoc
./cm/wf_gcp_cloud_md_get_projects.adoc
./cm/wf_onprem_get_wes.adoc
./cm/wf_gcp_cloud_md_get_disk_types.adoc
./cm/wf_cvo_before.adoc
./cm/wf_aws_ontap_create_vol_cifs.adoc
./cm/wf_aws_ontap_delete_cifs.adoc
./cm/wf_azure_ontap_delete_cifs.adoc
./cm/api_ref_resources.adoc
./cm/wf_azure_ontap_create_cifs.adoc
./cm/wf_onprem_ontap_delete_volume.adoc
./cm/wf_common_identity_get_nss_key.adoc
./cm/wf_azure_cloud_md_create_account.adoc
./cm/api_reference.adoc
./cm/wf_azure_cloud_md_get_vnets.adoc
./cm/wf_aws_ontap_get_volumes.adoc
./cm/cvo_deployment.adoc
./cm/wf_azure_cloud_md_get_permutations.adoc
./cm/wf_gcp_cloud_get_wes.adoc
./cm/wf_gcp_ontap_modify

In [ ]:
import os
import faiss
import numpy as np

In [ ]:

# Step 1: Collect .adoc files
adoc_files = []
adoc_count = 0

for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".adoc"):
            adoc_count += 1
            path = os.path.join(root, file)
            adoc_files.append(path)
            print("✅ Found:", path)

print(f"\nTotal .adoc files found: {adoc_count}")
print(f"\nTotal .adoc files found: {adoc_count}")
# Step 2: Read and chunk text
document_chunks = []
for filepath in adoc_files:
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
        # Basic chunking by paragraphs (or customize as needed)
        chunks = content.split('\n\n')  # you could also use spaCy for sentence chunking
        document_chunks.extend([chunk.strip() for chunk in chunks if chunk.strip()])


✅ Found: ./legal-notices.adoc
✅ Found: ./cm/wf_common_ontap_repl_get_rel_status.adoc
✅ Found: ./cm/wf_gcp_cloud_md_get_vpcs.adoc
✅ Found: ./cm/wf_aws_ontap_modify_volume.adoc
✅ Found: ./cm/wf_common_identity_get_supported_srv.adoc
✅ Found: ./cm/wf_azure_cloud_delete_we.adoc
✅ Found: ./cm/wf_aws_ontap_add_disks_aggr.adoc
✅ Found: ./cm/wf_aws_cloud_md_create_account.adoc
✅ Found: ./cm/wf_common_ontap_repl_get_rel_we_status.adoc
✅ Found: ./cm/prepare.adoc
✅ Found: ./cm/wf_gcp_cloud_md_get_projects.adoc
✅ Found: ./cm/wf_onprem_get_wes.adoc
✅ Found: ./cm/wf_gcp_cloud_md_get_disk_types.adoc
✅ Found: ./cm/wf_cvo_before.adoc
✅ Found: ./cm/wf_aws_ontap_create_vol_cifs.adoc
✅ Found: ./cm/wf_aws_ontap_delete_cifs.adoc
✅ Found: ./cm/wf_azure_ontap_delete_cifs.adoc
✅ Found: ./cm/api_ref_resources.adoc
✅ Found: ./cm/wf_azure_ontap_create_cifs.adoc
✅ Found: ./cm/wf_onprem_ontap_delete_volume.adoc
✅ Found: ./cm/wf_common_identity_get_nss_key.adoc
✅ Found: ./cm/wf_azure_cloud_md_create_account.adoc
✅ F

In [ ]:
print(len(document_chunks))         # Should be > 0
print(document_chunks[:2])          # Preview a few chunks


10164
['---\nsidebar: sidebar\npermalink: legal-notices.html\nkeywords: copyrights, notice, trademarks, patents, privacy, open source\nsummary: Legal notices provide access to copyright statements, trademarks, patents, and more.\n---', '= Legal notices\n:hardbreaks:\n:nofooter:\n:icons: font\n:linkattrs:\n:imagesdir: ./media/']


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast + effective
embeddings = model.encode(document_chunks, show_progress_bar=True)




Batches:   0%|          | 0/318 [00:00<?, ?it/s]

In [ ]:
# Convert to NumPy array
embedding_matrix = np.array(embeddings)

# Check and reshape if needed
if len(embedding_matrix.shape) == 1:
    embedding_matrix = embedding_matrix.reshape(1, -1)

print("Embedding matrix shape:", embedding_matrix.shape)

# Now safely create the FAISS index
import faiss
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)


Embedding matrix shape: (10164, 384)


In [ ]:
print(type(embeddings))
print(len(embeddings))         # Should match number of chunks
print(type(embeddings[0]))     # Should be a list or np.ndarray
print(np.array(embeddings).shape)  # Should be (N, D)


<class 'numpy.ndarray'>
10164
<class 'numpy.ndarray'>
(10164, 384)


In [ ]:
query = "AWS Status 200"

In [ ]:
query_embedding = model.encode([query]) #Encode


In [ ]:
# Search for the top 5 closest document chunks
D, I = index.search(np.array(query_embedding), k=5)


In [ ]:
for rank, idx in enumerate(I[0]):
    print(f"\n🔹 Match #{rank+1} — Distance: {D[0][rank]:.4f}")
    print(document_chunks[idx][:500])  # Preview first 500 characters


🔹 Match #1 — Distance: 0.8525
You must have AWS credentials.

🔹 Match #2 — Distance: 0.8811
==== Updates extra capacity serials
----
PUT /aws/ha/working-environments/{workingEnvironmentId}/extra-capacity-serials
----

🔹 Match #3 — Distance: 0.9359
Perform the workflow link:wf_aws_cloud_get_wes.html#get-working-environment-for-high-availability-pair[Get working environments] and choose the `publicId` and the `svmName` values for the `workingEnvironmentId` (working environment) and the `svmName` (SVM name) parameters. From the `capacityFeatures` field in the response, choose the value of `s3` for the `capacityTier` parameter.

🔹 Match #4 — Distance: 0.9380
Perform the workflow link:wf_aws_cloud_get_wes.html#get-working-environments-for-high-availability-pair[Get working environments] and choose the `publicId` and the `svmName` values for the `workingEnvironmentId`  and the `svmName` parameters. From the `capacityFeatures` field in the response, choose the value of `s3` for the `capacity